[![Binder](https://img2020.cnblogs.com/blog/2213660/202104/2213660-20210416003431953-219705653.png)](https://colab.research.google.com/github/monarch-beluga/Geemap_Tutorials/blob/master/Image/04_%E6%95%B0%E5%AD%A6%E8%BF%90%E7%AE%97.ipynb)

In [ ]:
# !pip install geemap

In [1]:
import geemap
import ee

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B5'),
      'RED': image.select('B4'),
      'BLUE': image.select('B2')
})

Map.centerObject(image, 8)
Map.addLayer(evi, {'min': -1, 'max': 1, 'palette': ['FF0000', '00FF00']})
Map

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])
# 普通方法
bare = ndvi.lt(0.2).And(ndwi.lt(0))

Map.setCenter(-122.3578, 37.7726, 11)
Map.addLayer(bare.updateMask(bare), {}, 'bare')
# expression方法
bare2 = ndvi.expression(
    'IT1 && IT2',{
        'IT1': ndvi.lt(0.2),
        'IT2': ndwi.lt(0)
    }
)
Map.addLayer(bare.updateMask(bare2), {}, 'bare2')

In [ ]:
# 普通方法
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones', opacity=0.5)
# expression方法
zonesExp = nl2012.expression(
    "(b('stable_lights') > 62) ? 3" +
      ": (b('stable_lights') > 55) ? 2" +
        ": (b('stable_lights') > 30) ? 1" +
          ": 0"
)
Map.addLayer(zonesExp,
             {'min': 0, 'max': 3, 'palette': palette},
             'development zones (ternary)', opacity=0.5)

In [ ]:
nl2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl2012.select('stable_lights')

zones = lights.where(lights.lte(30), 0).where(lights.gt(30), 1).where(lights.gt(55), 2).where(lights.gt(62), 3)


palette = ['000000', '0000FF', '00FF00', 'FF0000']
Map.setCenter(2.373, 48.8683, 8)
Map.addLayer(zones, {'min': 0, 'max': 3, 'palette': palette}, 'development zones', opacity=0.5)